In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()
output_null = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./info.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Kit_Part_Number'] != ''].drop_duplicates(['Kit_JOIN_TCPK'],
                                                                 keep='first',
                                                                 ignore_index=True)

print('总数量：' + str(len(input_)))
input_['Kit_JOIN_TCPK'].tolist()

总数量：926


['5724;10110008',
 '5724;10110012',
 '5728;10110016',
 '5728;10110020',
 '5736;10110028',
 '5736;10110032',
 '5736;10110036',
 '5728;10110040',
 '5724;8306420',
 '5723;8314716',
 '5736;10110024',
 '10465;10195124',
 '5724;7093400',
 '5728;8307288',
 '5736;8307904',
 '5736;8307908',
 '5736;8307912',
 '5723;8314756',
 '5724;10195068',
 '5728;10195092',
 '5722;10195120',
 '5722;10195128',
 '5728;10519392',
 '5728;10519400',
 '5728;10519428',
 '5736;10519432',
 '5736;10519444',
 '5724;12020477',
 '5724;12020481',
 '5724;10454488',
 '5728;10454548',
 '5736;10454552',
 '5728;10454556',
 '5722;10454620',
 '5723;10454624',
 '5724;8304564',
 '5724;8304568',
 '5728;8307540',
 '5728;8307544',
 '5728;8307548',
 '5728;8307552',
 '5736;8308268',
 '5736;8308276',
 '5723;8313592',
 '5728;8921324',
 '5724;8306272',
 '13625;8306340',
 '11976;8306344',
 '5728;8307348',
 '5728;8307356',
 '5728;8307360',
 '5736;8308096',
 '5723;8314724',
 '5722;8314728',
 '12016;10454544',
 '5724;8306428',
 '5568;8306432',

In [3]:
crawler_status = 'error'

crawler_status

error


In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

url = 'https://www.rockauto.com/en/parts/' + input_.loc[a, 'Kit_Manufacturer'].replace('+', '%2B').replace(',', '%2C') + ',' + input_.loc[a, 'Kit_Part_Number'] + ',Lennon,' + input_.loc[a, 'Kit_Type_Code']

resp = requests.get(url,
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'<!DOCTYPE HTML>\n<html class="no-js " lang="en">\n<head>\n<meta http-equiv="content-type" content="text/html; charset=windows-1252">\n<meta name="description" content="RockAuto ships auto parts and body parts from over 300 manufacturers to customers\' doors worldwide, all at warehouse prices. Easy to use parts catalog.">\n<meta name="keywords" content=",,,,,,">\n<meta name="Classification" content="automotive">\n<meta name="abstract" content="Online Store for brand name auto parts at warehouse-direct prices...">\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n\n<title></title>\n<link rel="stylesheet" type="text/css" href="/catalog/main-min.css?1718263456">\n<link rel="stylesheet" type="text/css" href="/catalog/reset-min.css?1718263456">\n<link rel="stylesheet" type="text/css" href="/catalog/mobile-min.css?1718263456">\n<link rel="styleshe

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./kit_part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE HTML>
<html class="no-js" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="RockAuto ships auto parts and body parts from over 300 manufacturers to customers' doors worldwide, all at warehouse prices. Easy to use parts catalog." name="description"/>
  <meta content=",,,,,," name="keywords"/>
  <meta content="automotive" name="Classification"/>
  <meta content="Online Store for brand name auto parts at warehouse-direct prices..." name="abstract"/>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <title>
  </title>
  <link href="/catalog/main-min.css?1718263456" rel="stylesheet" type="text/css"/>
  <link href="/catalog/reset-min.css?1718263456" rel="stylesheet" type="text/css"/>
  <link href="/catalog/mobile-min.css?1718263456" rel="stylesheet" type="text/css"/>
  <link href="/ca

In [8]:
manufacturer = html.xpath('//select[@id="manufacturer_partsearch_007"]/option[@selected]/text()')[0].strip()

manufacturer

IndexError: list index out of range

In [ ]:
manufacturer != input_.loc[a, 'Kit_Manufacturer']

In [ ]:
no_parts_found = html.xpath('//span[@class="navlabellink nvoffset nnormal"]/text()')

no_parts_found

In [ ]:
len(no_parts_found) != 0 and no_parts_found[0].strip() == 'No Parts Found'

In [ ]:
list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

list_part

In [ ]:
len(list_part)

In [ ]:
import json

In [ ]:
list_json = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_supplemental")]/@value')[0]) for part in list_part]

list_json

In [ ]:
list_json_2 = [json.loads(part.xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0]) for part in list_part]

list_json_2

In [ ]:
index_part = -1
for i in range(len(list_json)):
    if list_json[i]['catalogname'].strip() == input_.loc[a, 'Kit_Manufacturer'] and list_json[i]['partnumber'].strip() == input_.loc[a, 'Kit_Part_Number'] and list_json_2[i]['parttype'].strip() == input_.loc[a, 'Kit_Type_Code']:
        if index_part != -1:
            continue
        index_part = i
        
index_part

In [ ]:
category = html.xpath('//select[@id="partgroup_partsearch_007"]/option[@selected]/text()')[0].strip()

category

In [ ]:
type_ = html.xpath('//select[@id="parttype_partsearch_007"]/option[@selected]/text()')[0].strip()

type_

In [ ]:
list_note_1 = list_part[index_part].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

note_1

In [ ]:
list_note_2 = list_part[index_part].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')
note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

note_2

In [ ]:
src = list_part[index_part].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

src

In [ ]:
oe = list_part[index_part].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')
oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')
oe = ';'.join(sorted(list(set(oe.split(';'))), key=oe.split(';').index))

oe

In [ ]:
df_temp = pd.DataFrame([{'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK'],
                         'Kit_Category': category,
                         'Kit_Type': type_,
                         'Kit_Vehcile_Json': '',
                         'Kit_Vehcile': '',
                         'Kit_OE': oe,
                         'Kit_Note_1': note_1,
                         'Kit_Note_2': note_2,
                         'Kit_Pic': '',
                         'Kit_Url': url,
                         'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + input_.loc[a, 'Kit_Part_Key'],
                         'Kit_Src': src,
                         'Kit_Specifications_Json': '',
                         'Kit_Kit_Json': ''}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

In [ ]:
crawler_status = 'ok'

crawler_status

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Info': input_.loc[a, 'Info'],
                             'Kit_No.': int(input_.loc[a, 'Kit_No.']),
                             'Kit_Quantity': int(input_.loc[a, 'Kit_Quantity']),
                             'Kit_Note': input_.loc[a, 'Kit_Note'],
                             'Kit_Manufacturer': input_.loc[a, 'Kit_Manufacturer'],
                             'Kit_Part_Number': input_.loc[a, 'Kit_Part_Number'],
                             'Kit_Type_Code': input_.loc[a, 'Kit_Type_Code'],
                             'Kit_Part_Key': input_.loc[a, 'Kit_Part_Key'],
                             'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
if crawler_status == 'null':
    df_temp = pd.DataFrame([{'Kit_JOIN_TCPK': input_.loc[a, 'Kit_JOIN_TCPK'],
                             'Kit_Category': '',
                             'Kit_Type': '',
                             'Kit_Vehcile_Json': '',
                             'Kit_Vehcile': '',
                             'Kit_OE': '',
                             'Kit_Note_1': '',
                             'Kit_Note_2': '',
                             'Kit_Pic': '',
                             'Kit_Url': '',
                             'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + input_.loc[a, 'Kit_Type_Code'] + '&pk=' + input_.loc[a, 'Kit_Part_Key'],
                             'Kit_Src': '',
                             'Kit_Specifications_Json': '',
                             'Kit_Kit_Json': ''}])
    
    output_null = pd.concat([output_null, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./test_kit_part.xlsx', index=False)

output_correct

In [ ]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Info', 'Kit_No.'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./test_kit_part_error.xlsx', index=False)

output_error

In [ ]:
if len(output_null) != 0:
    output_null = output_null.drop_duplicates(ignore_index=True)
    output_null = output_null.sort_values(by=['Info', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
    output_null.to_excel('./test_kit_part_null.xlsx', index=False)

output_null